In [7]:
%run ../jupyter/notebook_modules.ipynb

In [1]:
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

In [2]:
client = language.LanguageServiceClient()

In [3]:
text = u'Hello, world!'
document = types.Document(
    content=text,
    type=enums.Document.Type.PLAIN_TEXT)

In [18]:
f = types.AnnotateTextRequest.Features()
f.extract_syntax = True

In [19]:
text = client.annotate_text(document=document, features=f)

In [24]:
text.tokens

[text {
  content: "Hello"
  begin_offset: -1
}
part_of_speech {
  tag: X
}
dependency_edge {
  head_token_index: 2
  label: DISCOURSE
}
lemma: "Hello"
, text {
  content: ","
  begin_offset: -1
}
part_of_speech {
  tag: PUNCT
}
dependency_edge {
  head_token_index: 2
  label: P
}
lemma: ","
, text {
  content: "world"
  begin_offset: -1
}
part_of_speech {
  tag: NOUN
  number: SINGULAR
}
dependency_edge {
  head_token_index: 2
  label: ROOT
}
lemma: "world"
, text {
  content: "!"
  begin_offset: -1
}
part_of_speech {
  tag: PUNCT
}
dependency_edge {
  head_token_index: 2
  label: P
}
lemma: "!"
]

^ cool, but too expensive for experimentation

In [2]:
%pip install spacy

    100% |████████████████████████████████| 29.7MB 45kB/s 
    100% |████████████████████████████████| 92kB 5.0MB/s 
    100% |████████████████████████████████| 3.2MB 365kB/s 
  Using cached https://files.pythonhosted.org/packages/9e/9b/62c60d2f5bc135d2aa1d8c8a86aaf84edb719a59c7f11a4316259e61a298/plac-0.9.6-py2.py3-none-any.whl
    100% |████████████████████████████████| 184kB 6.8MB/s 
    100% |████████████████████████████████| 2.1MB 664kB/s 
Note: you may need to restart the kernel to use updated packages.


In [38]:
%pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_md-2.1.0/en_core_web_md-2.1.0.tar.gz


    100% |████████████████████████████████| 95.4MB 13kB/s 
  Running setup.py bdist_wheel for en-core-web-md ... done
  Stored in directory: /home/jupyter/.cache/pip/wheels/c1/2c/5f/fd7f3ec336bf97b0809c86264d2831c5dfb00fc2e239d1bb01
Successfully built en-core-web-md
Note: you may need to restart the kernel to use updated packages.


In [1]:
import spacy

In [2]:
nlp = spacy.load("en_core_web_sm")

In [2]:
%pip install apache_beam[gcp]

UsageError: Line magic function `%pip` not found.


In [2]:
import apache_beam as beam
from apache_beam.runners.interactive import interactive_runner
from apache_beam.runners.portability import portable_runner
from apache_beam.options import pipeline_options
from apache_beam.options.pipeline_options import PipelineOptions, GoogleCloudOptions, StandardOptions

/home/jupyter/.local/lib/python3.5/site-packages/apache_beam/__init__.py:84: UserWarning: Running the Apache Beam SDK on Python 3 is not yet fully supported. You may encounter buggy behavior or missing features.
  'Running the Apache Beam SDK on Python 3 is not yet fully supported. '


In [2]:
options = PipelineOptions()
google_cloud_options = options.view_as(GoogleCloudOptions)
google_cloud_options.project = 'resonant-goods-241314'
google_cloud_options.job_name = 'spacy'
google_cloud_options.staging_location = 'gs://resonant-goods-241314/staging'
google_cloud_options.temp_location = 'gs://resonant-goods-241314/temp'
options.view_as(StandardOptions).runner = 'DataflowRunner'

In [3]:
options = PipelineOptions()
options.view_as(StandardOptions).runner = 'DirectRunner'
google_cloud_options = options.view_as(GoogleCloudOptions)
google_cloud_options.project = 'resonant-goods-241314'

In [3]:
p = beam.Pipeline(options=options)

In [4]:
(p 
 | beam.io.ReadFromText('gs://dataflow-samples/shakespeare/kinglear.txt')
 | 'ExtractWords' >> beam.FlatMap(lambda x: re.findall(r'[A-Za-z\']+', x)) 
 | beam.combiners.Count.PerElement()
 | beam.Map(lambda word_count: '%s: %s' % (word_count[0], word_count[1]))
 | beam.io.WriteToText('gs://resonant-goods-241314/counts.txt')
)


<PCollection[WriteToText/Write/WriteImpl/FinalizeWrite.None] at 0x7fee84a9cbe0>

In [5]:
result = p.run()
result.wait_until_finish()

DONE

In [8]:
import bq

importing Jupyter notebook from bq.ipynb
importing Jupyter notebook from competition.ipynb


In [11]:
read_spec = '{dataset_id}.train'.format(dataset_id=bq.dataset_id)
write_spec = '{dataset_id}.train_target_on_tokens'.format(dataset_id=bq.dataset_id)

In [12]:
table_schema = 'comment_text:STRING'

In [14]:
p = beam.Pipeline(options=options)

In [15]:
(
    p
    | 'ReadTable' >> beam.io.Read(beam.io.BigQuerySource(read_spec))
    # Each row is a dictionary where the keys are the BigQuery columns
    | beam.Map(lambda elem: elem['comment_text'])
    | beam.io.WriteToBigQuery(
        write_spec,
        schema=table_schema,
        write_disposition=beam.io.BigQueryDisposition.WRITE_TRUNCATE,
        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED)
)

<PDone[WriteToBigQuery/WriteToBigQuery/NativeWrite.None] at 0x7fee7fbe5128>

In [ ]:
import spacy
nlp = spacy.load('en_core_web_md')

In [5]:
documents = []
def collect(document):
    document['tokens'] = nlp(document['comment_text'])
    documents.append(document)
    return True

p = beam.Pipeline(options=options)

(
    p
    | 'ReadTable' >> beam.io.Read(beam.io.BigQuerySource(
        query='SELECT comment_text, target FROM '\
              '[resonant-goods-241314:jigsaw_unintended_bias_in_toxicity_classification.train]'\
              'LIMIT 10'
        ))
    | "collect" >> beam.Map(collect)
    #| beam.io.WriteToBigQuery(
    #    write_spec,
    #    schema=table_schema,
    #    write_disposition=beam.io.BigQueryDisposition.WRITE_TRUNCATE,
    #    create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED)
)

result = p.run()
result.wait_until_finish()

DONE

In [11]:
documents[0]['tokens'][0].norm_

probably